В этой тетрадке попробуем собрать все полученные знания в одном месте.
Обучаться будем на выборке кошек и собак.

Задача номер раз - собрать свою начальную сетку.

In [ ]:
from keras import layers
from keras import models
from keras import optimizers
model = models.Sequential()

model.summary()

Какие у нас должны быть шаги, в создании любой модели?

тут лежат нужные нам функции, для обучения 

In [2]:
from keras.preprocessing.image import ImageDataGenerator
import os
base_dir = 'cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')


# Задаем нормировку
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # говорим, где у нас все лежит
        train_dir,
        # Говорим-сделай нам все картинку в наше разрешение
        target_size=(150, 150),
        batch_size=20,
        # Нужен бинарный таргет, для нашей моедил
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Using TensorFlow backend.


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# в фит уже передаем созданные генераторы, во всем остальном обучение индентично
history = model.fit_generator(....)

In [ ]:
# не забываем сохранять модель
model.save('cats_and_dogs_small_1.h5')

Нарисуем процесс обучения

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

Data augementation

In [ ]:
# функции для прокрутки и сдвигов нашей картинки
datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

Тут добавляем аугументацию, смотрим, как меняется точность модели

In [ ]:

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

In [ ]:
history = model.fit_generator(....)
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

можно еще поиграться со сложностью модели или количество эпох.
Теперь заберем модельку, которые делали умные люди. И соеденим наше гениальное понимание и их.

## Feature extraction

Когда мы обучаем сверточные сети, то свертки пытаются найти патерны в данных и мы можем пытаться взять эту информацию из уже обученных сетей. Делается это примерно следующим образом

![swapping FC classifiers](https://s3.amazonaws.com/book.keras.io/img/ch5/swapping_fc_classifier.png)

Почему не использовать и полносвязные слои для такого подхода? Полносвязные слои по своей архитектуре учат слишком мелкие зависимости и могут использоваться только для изначальной задачи. Можно, конечно, попробавть использовать и переобучать и их, но значительно сложнее.

In [5]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(150, 150, 3))
# посмотрим на нашего монстра
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

In [ ]:
# загрузка данных
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory,conv_base, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Мы должны остановится, если подали хоть раз экземляр
            break
    return features, labels

train_features, train_labels = extract_features(train_dir,conv_base, 2000)
validation_features, validation_labels = extract_features(validation_dir,conv_base, 1000)
test_features, test_labels = extract_features(test_dir,conv_base, 1000)

In [ ]:
#Последний слой мы должны скормить полносвязнному слою 
#и наши картинки тоже должны быть в определенном формате. Воспользуемся этим.
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
# теперь ваша очередь -собираем и обучаем модель!
model = models.Sequential()

Смотрим на наш итог всего этого дела, стало ли нам лучше?

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

В первой части, мы собирали модель просто используя представления сети, но можем же собрать и полностью?

In [ ]:
from keras import models
from keras import layers

model = models.Sequential()


In [ ]:
print('Количество слоев, которое мы учим:', len(model.trainable_weights))

In [ ]:
# заморозим сверточную структуру
conv_base.trainable = False

In [ ]:
print('Количество слоев, которое мы учим после заморозки:', len(model.trainable_weights))

In [ ]:
# повторяем нашу дату генератор полностью, здесь дубль кода, чтобы далеко не ходить!

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Замечание! Не аугементриуем валидационные данные
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # директория, в которую пойдем забирать даннные
        train_dir,
        # Меняем shape наших картинок
        target_size=(150, 150),
        batch_size=20,
        # На нужны бинарные метки, так как юзаем кросс энтропию
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Fine-tuning

Также мы можем морозить только определенные слои, блоки. Что может быть удобным

![fine-tuning VGG16](https://s3.amazonaws.com/book.keras.io/img/ch5/vgg16_fine_tuning.png)

In [7]:
# модель выглядит следующим образом!
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

Почему мы отключаем последнии слои? чем сложнее модель, чем глубже слой, тем более сложные фичи она использует, и более обучается под специфичные данные. Также все упирается в данные и мошности, если мощностей нет или недостаточно данных, чем меньше мы параметров обучаем, тем нам лучше.

In [ ]:
# отключаем заморозку последнегшо слоя
conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
# пересобираем модель и рисуем картинку!

На выходе из данной тетрадки, мы научились всем шагам, которые нужны для построения сети:
1) Создаем сеть
2) Добавляем дата аугементатишн
3) используем представления сложной сети как входной параметр
3) Либо используем всю сеть и её дообучаем(дольше и сложнее, но может дать больший профит)
